# Tortoise TTS (Original Repository)

This notebook is designed for [@fakerybakery's guide](https://github.com/fakerybakery/tortoise-tts-guide) on running and fine-tuning Tortoise TTS.

Before you begin, I **strongly** recommend you turn on a GPU runtime.

In [ ]:
!git clone https://github.com/neonbjb/tortoise-tts
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install -e .
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip3 install deepspeed==0.9.0
!pip install transformers==4.29.2 # to fix bug

In [9]:
#@title # Setup
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

In [11]:
#@markdown This is the text that will be spoken. If you want longform audio, ignore the `text` element - we'll manually set it later. But still run this cell for the preset!

text = "Joining two modalities results in a surprising increase in generalization! What would happen if we combined them all?" #@param {type:"string"}
#@markdown Show code for multiline text input
# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast" (default), "fast", "standard", "high_quality"}. See docs in api.py
preset = "ultra_fast" #@param ["ultra_fast", "fast", "standard", "high_quality"]

In [ ]:
#@markdown Tortoise will attempt to mimic voices you provide. It comes pre-packaged
#@markdown with some voices you might recognize.

#@markdown Let's list all the voices available. These are just some random clips I've gathered
#@markdown from the internet as well as a few voices from the training dataset.
#@markdown Feel free to add your own clips to the voices/ folder.
#%cd tortoise-tts-fast
%ls tortoise/voices
import IPython
IPython.display.Audio(filename='tortoise/voices/tom/1.wav')

## Long Audio

For short audio, skip to the short section.

In [ ]:
text = """
Long text here
"""

In [ ]:
!pip3 install nltk
import nltk
nltk.download('punkt')

In [ ]:
script = text.replace("\n", " ").strip()
sentences = nltk.sent_tokenize(script)

voice = 'train_dotrice' # select voice

audio_list = []
voice_samples, conditioning_latents = load_voice(voice)
for txt in sentences:

    #@markdown Load it and send it through Tortoise.
    gen = tts.tts_with_preset(txt, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset, k=1)
    audio_list.append(gen.squeeze(0).cpu())
concatenated_audio = torch.cat(audio_list, dim=1)
torchaudio.save('generated.wav', concatenated_audio, 24000)
IPython.display.Audio('generated.wav')

## Short Audio

In [ ]:
#@markdown Pick one of the voices from the output above
voice = 'train_dotrice' #@param {type:"string"}
text = 'Text' #@param {type:"string"}

#@markdown Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [ ]:
#@markdown Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
#@markdown (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [14]:
#@markdown Optionally, upload use your own voice by running the next two cells. I recommend
#@markdown you upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.
CUSTOM_VOICE_NAME = "custom"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

In [ ]:
# Generate speech with the custotm voice.
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['pat', 'william'])

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.", 
                          voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')